In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Set the path to the data
data_dir = '/content/drive/My Drive/SRI'
csv_file = '/content/drive/My Drive/SRI/english.csv'


In [ ]:
# Load the CSV file
data = pd.read_csv(csv_file)

data

,image,label
0,Img/img001-001.png,0
1,Img/img001-002.png,0
2,Img/img001-003.png,0
3,Img/img001-004.png,0
4,Img/img001-005.png,0
...,...,...
3405,Img/img062-051.png,z
3406,Img/img062-052.png,z
3407,Img/img062-053.png,z
3408,Img/img062-054.png,z


In [ ]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(data['image'], data['label'], test_size=0.2, random_state=42)


In [ ]:
# Preprocess the data
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_dataframe(
    dataframe=data.loc[X_train.index],
    directory=data_dir,
    x_col="image",
    y_col="label",
    target_size=(400, 300),
    batch_size=32,
    class_mode='categorical'
)

Found 2728 validated image filenames belonging to 62 classes.


In [ ]:
val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_dataframe(
    dataframe=data.loc[X_val.index],
    directory=data_dir,
    x_col="image",
    y_col="label",
    target_size=(400, 300),
    batch_size=32,
    class_mode='categorical'
)


Found 682 validated image filenames belonging to 62 classes.


In [ ]:
# Load the pre-trained DenseNet121 model
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(400, 300, 3))


29084464/29084464 [==============================] - 0s 0us/step


In [ ]:
# Freeze the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
# Add custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
output = Dense(62, activation='softmax')(x)

In [ ]:
model = Model(inputs=base_model.input, outputs=output)


In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Train the model
model.fit(train_generator, epochs=20, validation_data=val_generator)

Epoch 1/20
86/86 [==============================] - 1781s 20s/step - loss: 3.3998 - accuracy: 0.2350 - val_loss: 2.6367 - val_accuracy: 0.4619
Epoch 2/20
86/86 [==============================] - 61s 714ms/step - loss: 2.1331 - accuracy: 0.6250 - val_loss: 1.8735 - val_accuracy: 0.6188
Epoch 3/20
86/86 [==============================] - 61s 705ms/step - loss: 1.5474 - accuracy: 0.7119 - val_loss: 1.4845 - val_accuracy: 0.6921
Epoch 4/20
86/86 [==============================] - 70s 821ms/step - loss: 1.2227 - accuracy: 0.7760 - val_loss: 1.2470 - val_accuracy: 0.7273
Epoch 5/20
86/86 [==============================] - 62s 717ms/step - loss: 1.0311 - accuracy: 0.8054 - val_loss: 1.1283 - val_accuracy: 0.7390
Epoch 6/20
86/86 [==============================] - 61s 709ms/step - loss: 0.9047 - accuracy: 0.8207 - val_loss: 1.0250 - val_accuracy: 0.7610
Epoch 7/20
86/86 [==============================] - 61s 711ms/step - loss: 0.8031 - accuracy: 0.8354 - val_loss: 0.9481 - val_accuracy: 0.7683

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(
    dataframe=data.loc[X_val.index],
    directory=data_dir,
    x_col="image",
    y_col="label",
    target_size=(400, 300),
    batch_size=32,
    class_mode='categorical'
)

test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Accuracy: {test_accuracy:.4f}')

Found 682 validated image filenames belonging to 62 classes.
22/22 [==============================] - 13s 567ms/step - loss: 0.6178 - accuracy: 0.8255
Test Accuracy: 0.8255


In [ ]:
import joblib

joblib.dump(model, '/content/drive/My Drive/SRI/DenseNet121_model.pkl')

['/content/drive/My Drive/SRI/DenseNet121_model.pkl']